# Subdatasets Meta-Learning Distribution

In [ ]:
import os
from os.path import join

import numpy as np
import pandas as pd

from common import MCLASSES, filter_msets, read_toml, save_toml


In [ ]:
def filter_dataset(df, filter_df, mclasses, dataset, datasets):
    other_datasets = [ds for ds in datasets if ds != dataset]
    tdf = df[df[mclasses['mtst']].any(axis=1)]
    tdf = tdf[tdf['dataset'].isin(other_datasets)]
    exclude = tdf.index
    filter_df.loc[exclude, 'mtst'] = 0
    return filter_df

In [ ]:
metachest_dir = read_toml('config.toml')['metachest_dir']
mtl_dir = join(metachest_dir, 'mtl')
os.makedirs(mtl_dir, exist_ok=True)

mclasses = MCLASSES
df = pd.read_csv(join(metachest_dir, 'metachest.csv'))


datasets = list(df['dataset'].unique())
for dataset in datasets:

    filter_df = pd.DataFrame(
        np.ones([df.shape[0], 3], dtype=int),
        columns=mclasses.keys()
    )
    filter_msets(df, filter_df, mclasses)
    filter_dataset(df, filter_df, mclasses, dataset, datasets)

    name = join(mtl_dir, f'ds_{dataset}')
    save_toml(f'{name}.toml', mclasses)
    filter_df.to_csv(f'{name}.csv', index=False)